In [ ]:
"""
This code was created by Dr. Antonino Cucchiara and Máire Volz. For this internship I was called to build upon their work and use this code to create a new map of all Fellows at NASA past and present.


"""

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import folium
import json
from folium import plugins
#Department Map Making code below:

FellowsFile = '/content/drive/MyDrive/NASA Fellows Database Internship/FellowswithNumData(all).csv'
MSIR2File = '/content/drive/MyDrive/NASA Fellows Database Internship/FINAL_MSIR2Data.csv'

In [ ]:
def mapMaker(FellowsFile, MSIR2File):
    '''
    This function takes in the necessary files to build the NHFP Map with surrounding insitutions.
    All inputs are string file names or directory maps.

    EDIT BELOW:
    FellowsFile = csv containing fellow information ('allFellows_FellowshipTypeSplit.csv')
    MSIR2File = csv containing information on HSIs ('FINAL_MSIR2Data.csv')

    EDIT BELOW:
    This function returns an interactive map.
    '''

    # STEP 1: Read in files
    allFellows = pd.read_csv(FellowsFile)
    MSIs = pd.read_csv(MSIR2File)

    HSIs = MSIs[MSIs['Classification'] == 'HSI']
    HSIs = HSIs[HSIs['Astronomy Program (Y/N)'] != 'N']

    HBCUs = MSIs[MSIs['Classification'] == 'HBCU']
    HBCUs = HBCUs[HBCUs['Astronomy Program (Y/N)'] != 'N']

    BSIs = MSIs[MSIs['Classification'] == 'BSI']
    BSIs = BSIs[BSIs['Astronomy Program (Y/N)'] != 'N']

    R2s = MSIs[MSIs['Research Classification'] == 'R2']
    R2s = R2s[R2s['Classification'] != 'HSI']
    R2s = R2s[R2s['Classification'] != 'BSI']
    R2s = R2s[R2s['Classification'] != 'HBCU']
    R2s = R2s[R2s['Astronomy Program (Y/N)'] != 'N']

    # STEP 2: Initialize map
    usMap = folium.Map(location=[39.0,-96.0], tiles = 'CartoDB Positron', zoom_start=4, control = False, name = "Base Map")

    tit = 'Past NHFP Host Institutions and Nearby Institutions with Astronomy Programs'
    title_html = '''
                 <h3 align="center" style="font-size:24px"><b>{}</b></h3>
                 '''.format(tit)

    sig = 'Credit: M. Volz, A. Cucchiara (NASA HQ)'
    sig_html = '''
                <h1 align="center" style = "font-size:16px">{}</h1>
                '''.format(sig)

    usMap.get_root().html.add_child(folium.Element(title_html))
    usMap.get_root().html.add_child(folium.Element(sig_html))

    # STEP 3: Build Fellow layer
    FG_Fellows = folium.FeatureGroup(name = 'Fellows')

    for i in range(len(allFellows)):
        try:
            folium.CircleMarker(
                location = [allFellows['Latitude'][i], allFellows['Longitude'][i]],
                popup = allFellows['Host Institution'][i] + '. ' + str(allFellows['Number of Fellows'][i]) + ' Fellows in total.' + \
        ' H: ' + str(allFellows['Number of Hubble Fellows'][i]) + \
        ' S: ' + str(allFellows['Number of Sagan Fellows'][i]) + \
        ' E: ' + str(allFellows['Number of Einstein Fellows'][i]) + \
        ' C: ' + str(allFellows['Number of Chandra Fellows'][i]) + \
        ' F: ' + str(allFellows['Number of Fermi Fellows'][i]) + \
        ' M: ' + str(allFellows['Number of Michelson Fellows'][i]) + \
        ' Sp: ' + str(allFellows['Number of Spitzer Fellows'][i]),
                radius = float(allFellows['Number of Fellows'][i]),
                color = 'crimson',
                fill = True,
                fill_color = 'crimson'
            ).add_to(FG_Fellows)
        except:
            i = i+1

    FG_Fellows.add_to(usMap)

    # STEP 4: Build BSI layer
    FG_BSIs = folium.FeatureGroup(name = 'Black Serving Institutions')

    for i in range(len(BSIs)):
        try:
            folium.Marker(
                location = [BSIs.values[i][9], BSIs.values[i][10]],
                popup = BSIs.values[i][0],
                icon = folium.Icon(color = 'darkpurple', icon_color = 'white', icon = 'circle'),
                fill = True,
            ).add_to(FG_BSIs)
        except:
            i = i+1

    FG_BSIs.add_to(usMap)

    # STEP 5: Build HSI Layer
    FG_HSIs = folium.FeatureGroup(name = 'Hispanic Serving Institutions')

    for i in range(len(HSIs)):
        try:
            folium.Marker(
                location = [HSIs.values[i][9], HSIs.values[i][10]],
                popup = HSIs.values[i][0],
                icon = folium.Icon(color = 'blue', icon_color = 'white', icon = 'circle'),
                fill = True,
            ).add_to(FG_HSIs)
        except:
            i = i+1

    FG_HSIs.add_to(usMap)

    # STEP 6: Build HBCU layer
    FG_HBCUs = folium.FeatureGroup(name = 'HBCUs')

    for i in range(len(HBCUs)):
        try:
            folium.Marker(
                location = [HBCUs.values[i][9], HBCUs.values[i][10]],
                popup = HBCUs.values[i][0],
                icon = folium.Icon(color = 'darkblue', icon_color = 'white', icon = 'circle'),
                fill = True,
            ).add_to(FG_HBCUs)
        except:
            print(i)

    FG_HBCUs.add_to(usMap)

    FG_R2s = folium.FeatureGroup(name = 'R2 Institutions')

    # STEP 7: Build R2 layer
    for i in range(len(R2s)):
        try:
            folium.Marker(
                location = [R2s.values[i][9], R2s.values[i][10]],
                popup = R2s.values[i][0],
                icon = folium.Icon(color = 'orange', icon_color = 'white', icon = 'circle'),
                fill = True,
            ).add_to(FG_R2s)
        except:
            i = i+1

    FG_R2s.add_to(usMap)

    # STEP 8: Add legend to map
    folium.LayerControl(collapsed = False).add_to(usMap)

    # STEP 9: Download map
    usMap.save('/content/drive/MyDrive/NASA Fellows Database Internship/NHFPFellowsMap22-24.html')

    return usMap

In [ ]:
mapMaker(FellowsFile, MSIR2File)

In [ ]:
usMap = mapMaker(FellowsFile, MSIR2File)
usMap.save('/content/drive/MyDrive/NASA Fellows Database Internship/NHFPFellowsMap22-24.html')

In [ ]:
import pandas as pd

# List of CSV file paths
csv_files = ['/content/drive/MyDrive/NASA Fellows Database Internship/HubbleFellows_numInst.csv', '/content/drive/MyDrive/NASA Fellows Database Internship/SaganFellows_numInst.csv', '/content/drive/MyDrive/NASA Fellows Database Internship/EinsteinFellows_numInst.csv', '/content/drive/MyDrive/NASA Fellows Database Internship/FellowswithNumData23-24.csv']

# Read and concatenate CSV files into a single DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Remove duplicate entries
combined_df.drop_duplicates(inplace=True)

# Save the combined DataFrame to a new CSV file
save_path = '/content/drive/MyDrive/NASA Fellows Database Internship/combined.csv'

# Save the combined DataFrame to the specified location
combined_df.to_csv(save_path, index=False)

# Print the combined DataFrame
print(combined_df)

print("Combined CSV file created successfully without duplicates.")


     Unnamed: 0                      Host Institution   Latitude   Longitude  \
0           0.0    California Institute of Technology  34.137014 -118.125288   
1           2.0  University of California Los Angeles  34.070878 -118.446850   
2           3.0               University of Minnesota  44.977803  -93.233251   
3           4.0                  University of Hawaii  21.332066 -157.921888   
4           5.0  National Radio Astronomy Observatory  17.756553  -64.583913   
..          ...                                   ...        ...         ...   
185         NaN                University of Michigan  42.294214  -83.710039   
186         NaN         University of Texas at Austin  30.285149  -97.733935   
187         NaN                    University of Utah  40.762814 -111.836872   
188         NaN                University of Virginia  38.041058  -78.505508   
189         NaN                Wayne State University  42.357205  -83.070950   

     Number of Fellows  Number of Hubbl

In [ ]:
import pandas as pd

# Replace 'path/to/your/file.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/NASA Fellows Database Internship/host and flavor.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Assuming your DataFrame has columns named 'University' and 'Fellowship Type'
# Group the DataFrame by 'University' and count the occurrences of each fellowship type
fellowship_counts = df.groupby('Host Institution')['Fellowship Type'].value_counts().unstack(fill_value=0)

# Save the resulting DataFrame to a new CSV file
output_file_path = '/content/drive/MyDrive/NASA Fellows Database Internship/fellowship_counts.csv'
fellowship_counts.to_csv(output_file_path)

print(f"Fellowship counts saved to {output_file_path}")

Fellowship counts saved to /content/drive/MyDrive/NASA Fellows Database Internship/fellowship_counts.csv
